# OSM

In [ ]:
from geopy.geocoders import Nominatim
import sqlalchemy
import geocoder
import pandas as pd
import numpy as np
from time import sleep

geolocator = Nominatim(user_agent='osm', timeout=60)
server_name = ''

conn = sqlalchemy.create_engine(
    f'mssql+pyodbc://'+server_name+'/kazakhstan?trusted_connection=yes&driver=ODBC Driver 17 for SQL Server')

In [ ]:
q = """select one.id, one.name_rus as town,
            three.name_rus + ' ' + lower(t.value_ru) as county from ats one
            left join ats two on one.parent_id = two.id
            left join ats three on two.parent_id = three.id
            left join ats_types t on three.d_ats_type_id = t.id
            where one.cato like '43%'
            and one.id in (select id from kyzylorda_ob_ats_osm
            where state = 'nan')
            order by one.id"""

df = pd.read_sql_query(q, conn)

In [ ]:
df['county'] = ['Кызылординская область' if df.loc[i, 'county'] in ('Казахстан республика', None) 
                else df.loc[i, 'county'] for i in list(df.index)]
df['town'] = ['' if df.loc[i, 'town'] == 'Кызылординская' else df.loc[i, 'town'] for i in list(df.index)]
df['county'] = ['Кызылорда' if 'город областного значения' in df.loc[i, 'county'] 
                else df.loc[i, 'county'] for i in list(df.index)]
df['town'] = [i.replace('ский', '') for i in df['town']]

In [ ]:
# df['name_rus'] = [df.loc[i, 'name_rus'].split(' ')[-1] for i in list(df.index)]
df['name_rus'] = [df.loc[i, 'name_rus'].split(' ')[-1] if df.loc[i, 'name_rus'][-2:] in ('ов', 'ва', 'ев') 
                  else df.loc[i, 'name_rus'] for i in list(df.index)]

In [ ]:
df.head()

In [ ]:
from geopy.point import Point
def get_osm(i, df):
    full_address = df.loc[i, 'town'] + ', ' + df.loc[i, 'county']
    g = geolocator.geocode(full_address, 
                       addressdetails=True, language='ru', country_codes='KAZ')
    return g.raw if pd.notnull(g) else np.nan

In [ ]:
cols = ['id', 'osm_type', 'lat', 'lng', 'class', 'type', 'hamlet', 'village', 
        'town', 'county', 'state', 'city', 'postcode', 'country_code']

In [ ]:
table_name = 'kyz_ats2'

dtype = {
    'id': sqlalchemy.types.NVARCHAR(),
    'osm_type': sqlalchemy.types.NVARCHAR(),
    'lat': sqlalchemy.types.NVARCHAR(),
    'lng': sqlalchemy.types.NVARCHAR(),
    'class': sqlalchemy.types.NVARCHAR(),
    'type': sqlalchemy.types.NVARCHAR(),
    'village': sqlalchemy.types.NVARCHAR(),
    'hamlet': sqlalchemy.types.NVARCHAR(),
    'town': sqlalchemy.types.NVARCHAR(),
    'county': sqlalchemy.types.NVARCHAR(),
    'state': sqlalchemy.types.NVARCHAR(),
    'city': sqlalchemy.types.NVARCHAR(),
    'postcode': sqlalchemy.types.NVARCHAR(),
    'country_code': sqlalchemy.types.NVARCHAR()
}

for i in list(df.index):
    geocode = get_osm(i, df)
    geocoded_df = pd.DataFrame(columns=cols, dtype='object')
    geocoded_df.loc[i, 'id'] = df.loc[i, 'id']
    if pd.isnull(geocode):
        geocoded_df.astype(str).to_sql(table_name, conn, if_exists='append', index=False, dtype=dtype)
        print(i, geocoded_df.loc[i, 'id'])
        continue
    geocoded_df.loc[i, 'osm_type'] = geocode['osm_type']
    geocoded_df.loc[i, 'lat'] = geocode['lat']
    geocoded_df.loc[i, 'lng'] = geocode['lon']
    geocoded_df.loc[i, 'class'] = geocode['class']
    geocoded_df.loc[i, 'type'] = geocode['type']
    if 'hamlet' in geocode['address'].keys():
        geocoded_df.loc[i, 'hamlet'] = geocode['address']['hamlet']
    if 'village' in geocode['address'].keys():
        geocoded_df.loc[i, 'village'] = geocode['address']['village']
#     if 'road' in geocode['address'].keys():
#         geocoded_df.loc[i, 'road'] = geocode['address']['road']
#         if 'building' in geocode['address'].keys() and not f['address']['building'].isdigit():
#             geocoded_df.loc[i, 'road'] += ' ' + geocode['address']['building']
    if 'town' in geocode['address'].keys():
        geocoded_df.loc[i, 'town'] = geocode['address']['town']
    if 'county' in geocode['address'].keys():
        geocoded_df.loc[i, 'county'] = geocode['address']['county']
    if 'state' in geocode['address'].keys():
        geocoded_df.loc[i, 'state'] = geocode['address']['state']
    if 'city' in geocode['address'].keys():
        geocoded_df.loc[i, 'city'] = geocode['address']['city']
    if 'postcode' in geocode['address'].keys():
        geocoded_df.loc[i, 'postcode'] = geocode['address']['postcode']
    if 'country_code' in geocode['address'].keys():
        geocoded_df.loc[i, 'country_code'] = geocode['address']['country_code']
        
    geocoded_df.astype(str).to_sql(table_name, conn, if_exists='append', index=False, dtype=dtype)
    print(i, geocoded_df.loc[i, 'id'])
    sleep(1)

In [ ]:
df = df[df.index > 40]

In [ ]:
q = """select one.id, one.name_rus as town,
            three.name_rus + ' ' + lower(t.value_ru) as county from ats one
            left join ats two on one.parent_id = two.id
            left join ats three on two.parent_id = three.id
            left join ats_types t on three.d_ats_type_id = t.id
            where one.cato like '43%'
            and one.id in (select id from kyzylorda_ats
            where lat =  'nan')
            order by one.id"""
df = pd.read_sql_query(q, conn)

# q2 = """select *from kyzylorda_ob_ats_osm"""
# ats = pd.read_sql_query(q2, conn)